## 03.Boosting계열
Boosting은 **여러 개의 약한 모델(Weak Learner 주로 얕은 결정 트리)** 을 **순차적으로 학습**시켜 **이전 모델의 오류를 보완**해나가는 **강한 예측기(Strong Predicter)** 를 사용하는 앙상블 기법이다.


- 약한 모델(예: 작은 깊이의 결정 트리)을 순차적으로 학습
- 이전 모델의 **오류(잔차)** 를 다음 모델이 보정(이전 모델이 틀린 샘플에 더 높은 가중치)
- 각 모델의 예측을 **가중합**하여 최종 예측을 수행
- 학습 방식: **순차적** / **오류 중심 개선**
- 사용 모델: 주로 **깊이가 얕은 결정 트리**
- 최적화 방식: **경사하강법(Gradient Descent)**


| 장점 | 단점 |
|------|------|
| 과적합에 비교적 강함 | 트리를 **순차적으로 학습**하여 속도가 느림 |
| 일반적으로 랜덤 포레스트보다 성능이 더 높음 | 병렬 처리 어려움 (순차 학습 구조) |




**Boosting 알고리즘 발전 단계**
- Boosting은 **오차 보정 방식의 순차적 학습 구조**로 앙상블을 구성
- **Gradient Boosting**은 경사하강법으로 트리를 추가하며 오차를 줄임
- **HistGradientBoosting**은 학습 속도와 메모리 효율성을 높인 개선된 버전 (노드분할 효율성을 위해 연속형 특성에 대한 binning 처리)






**손실 함수 및 학습 설정**


| 항목 | 설명 |
|------|------|
| **분류** | 로지스틱 손실 함수(log loss) 사용 |
| **회귀** | 평균제곱오차(MSE) 사용 |
| **학습률 (learning_rate)** | 한 번에 추가되는 트리의 영향력을 조절 |
| **subsample** | 훈련 세트 중 일부만 사용하면 확률적 경사하강법(SGD)처럼 작동 |


> ⚠ `subsample < 1.0` → **일부 데이터로 학습하여 일반화 성능 향상 가능**




**주요 알고리즘 및 클래스**


알고리즘 | 등장년도 | 핵심 특징
--- | --- | ---
AdaBoost | 1996 | 가중치를 이용해 오답 샘플을 강조
Gradient Boosting (GBM) | 1999 | 잔차(오차)를 줄이도록 새 모델 학습
XGBoost | 2014 | 정규화 + 병렬처리 최적화
LightGBM | 2017 | 히스토그램 기반 + Leaf-wise 트리
CatBoost | 2017 | 범주형 자동 처리 + 빠른 속도
HistGradientBoosting (scikit-learn) | 2019~ | LightGBM과 유사한 히스토그램 기반 구현






**히스토그램 기반 그레디언트 부스팅이란?**


히스토그램 기반 그레디언트 부스팅은 기존 GBM의 **느린 학습 속도** 문제를 개선한 방식이다.


- 연속 데이터를 **구간(binning)** 으로 나눠 계산량을 줄임
- 속도는 빠르면서도 일반 GBM에 가까운 예측 성능 유지
- 대용량 데이터 처리에 적합




In [50]:
from statistics import LinearRegression
from turtledemo.sorting_animate import disable_keys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import VotingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import root_mean_squared_error, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

## 01. SimpleGradientBoostingClassifier  구현
- 잔차를 학습하는 GradientBoosting 분류기를 직접 구현

In [51]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()
# 데이터 분리
X = cancer.data
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print(X_train[:3])
print(y_train[:3])

(455, 30) (455,)
(114, 30) (114,)
[[1.032e+01 1.635e+01 6.531e+01 3.249e+02 9.434e-02 4.994e-02 1.012e-02
  5.495e-03 1.885e-01 6.201e-02 2.104e-01 9.670e-01 1.356e+00 1.297e+01
  7.086e-03 7.247e-03 1.012e-02 5.495e-03 1.560e-02 2.606e-03 1.125e+01
  2.177e+01 7.112e+01 3.849e+02 1.285e-01 8.842e-02 4.384e-02 2.381e-02
  2.681e-01 7.399e-02]
 [2.018e+01 1.954e+01 1.338e+02 1.250e+03 1.133e-01 1.489e-01 2.133e-01
  1.259e-01 1.724e-01 6.053e-02 4.331e-01 1.001e+00 3.008e+00 5.249e+01
  9.087e-03 2.715e-02 5.546e-02 1.910e-02 2.451e-02 4.005e-03 2.203e+01
  2.507e+01 1.460e+02 1.479e+03 1.665e-01 2.942e-01 5.308e-01 2.173e-01
  3.032e-01 8.075e-02]
 [1.066e+01 1.515e+01 6.749e+01 3.496e+02 8.792e-02 4.302e-02 0.000e+00
  0.000e+00 1.928e-01 5.975e-02 3.309e-01 1.925e+00 2.155e+00 2.198e+01
  8.713e-03 1.017e-02 0.000e+00 0.000e+00 3.265e-02 1.002e-03 1.154e+01
  1.920e+01 7.320e+01 4.083e+02 1.076e-01 6.791e-02 0.000e+00 0.000e+00
  2.710e-01 6.164e-02]]
[1 0 1]


In [52]:
# 이진 분류
 # - sigmoid(z) : 숫자값을 인자로 받아 확률값 반환
 # - log_odds(p) : 확률값을 인자로 받아 log(숫자값 z)로 반환

 # 초기값
y_pred  = np.mean(y_train)
print('y_pred', y_pred) # 양성클래스 확률값

# 로그오즈 계산 :  입력(p: 확률) -> 출력(z : 로그오즈)
log_odds = lambda p: np.log(p/(1-p))
z = log_odds(y_pred)
print('z=', z)

# 시그모이드 계산 : 입력 (z: 로그 오즈) -> 출력(p: 확률)
sigmoid = lambda z:1 / (1+np.exp(-z))
p = sigmoid(z)
print('p=', p)

y_pred 0.6263736263736264
z= 0.5166907432183888
p= 0.6263736263736264


In [53]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

class SimpleGradientBoostingClassifier:
    def __init__(self, n_estimators=100, lr=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.lr = lr
        self.max_depth = max_depth

        self.trees = [] # 약학습기를 관리할 리스트
        self._inital_log_odds = 0 #로그오즈 초기값

    def log_odds(self, p):
        """p(확률)을 입력 받아, z(로그오즈)를 반환하는 함수)"""
        return np.log(p/(1-p))


    def sigmoid(self, z):
        """z(로그오즈)을 입력 받아, p(확률)를 반환하는 함수)"""
        return 1/(1 + np.exp(-z))

    def fit(self, X, y):
        """모델 학습 함수"""
        #부스팅은 기존의 모델이랑 새로운것을 더해야하기 때문에 초기로그오즈값이 필요하다.

        # 초기 로그오즈값 설정 (y 평균값)
        y_mean = np.mean(y) # 확률값, 꼭 평균값일 필요는 없는데 손실을 최소화 해줘서 보통 사용한다.
        self.initial_log_odds = self.log_odds(y_mean) # 확률값 -> 로그오즈

        y_pred_log_odds = np.full_like(y, self.initial_log_odds, dtype = np.float64)
        #print('y_pred_log_odds', y_pred_log_odds)

        # 반복적으로 약학습기(회귀예측기)를 순차적으로 학습 : 이전 오차를 학습
        for _ in range(self.n_estimators):
            # 오차를 계산하기 위하여
            # 로그오즈값(z) -> 확률값(p) 변환 : 왜 ? 오차를 구하기 위해서
            y_pred_proba = self.sigmoid(y_pred_log_odds)
                                # 확률값변환 여기서

            # 잔차(오차) 계산
            residuals = y - y_pred_proba

            # 잔차에 대한 학습
            dt_reg = DecisionTreeRegressor(max_depth=self.max_depth)
            dt_reg.fit(X, residuals) # 이전 오차에 대한 것을 학습하기 때문에 -> 그러면 어떻게 되냐? 오차를 보고 얼마나 보정을 해야하는 지 알 수 있음


            # 현재 예측값으로 로그오즈값 갱신
            y_pred = dt_reg.predict(X) # 보정량 예측한 값
            y_pred_log_odds -= self.lr * y_pred # 보정량을 조금씩 쪼개서 적용시킨다.
                                # 잔차에 대해서 학습을 했으면 얼마나 보정해야할지 보정량이 나올 것이고
                                # 보정량을 몇 퍼센트 고정할 지 조금씩
            # 약학습기 관리
            self.trees.append(dt_reg) # 잔차가 계속 줄어드는 방식으로 반복된다.



    def predict_proba(self, X):
        """모델 예측을 위한 확률값 계산 함수"""

        # 초기 로그오즈 설정
        y_pred_log_odds = np.full((X.shape[0],), self.initial_log_odds,
                                  dtype = np.float64)

        # 학습된 약학습기의 예측 결과
        for tree in self.trees: # 잔차를 학습한 회귀 트리 반복
            y_pred_log_odds += self.lr * tree.predict(X) # 이미 잔차 방향으로 정해져있기 때문에 +를 해준다. (!= 위에서 fit 할 때 (오차 반대방향으로 간다. 손실을 줄이기 위해서))

        # 로그오즈값을 확률값으로 변환해서 반환
        return self.sigmoid(y_pred_log_odds) # 양성클래스의 확률값만 반환

    def predict(self, X):
        """모델 예측 함수"""
        return (self.predict_proba(X) >= 0.5).astype(int)


sgb_clf = SimpleGradientBoostingClassifier()
sgb_clf.fit(X_train, y_train)


# 평가
y_pred = sgb_clf.predict(X_train)
print("Train Accuracy :", accuracy_score(y_train, y_pred))


y_pred = sgb_clf.predict(X_test)
print("Test Accuracy :", accuracy_score(y_test, y_pred))

Train Accuracy : 0.9736263736263736
Test Accuracy : 0.9385964912280702


# 02. GradientBoostingClassifier

In [54]:
# 이진 분류 : 유방암 진단
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(random_state=42, max_depth=3)
gb_clf.fit(X_train, y_train)

y_pred = gb_clf.predict(X_train)
print("Train Accuracy :", accuracy_score(y_train, y_pred))

y_pred = gb_clf.predict(X_test)
print("Test Accuracy :", accuracy_score(y_test, y_pred))

Train Accuracy : 1.0
Test Accuracy : 0.956140350877193


In [55]:
# 다중분류
from sklearn.datasets import load_wine


wine = load_wine()
X, y = wine.data, wine.target

# 타겟 클래스 0 1 2
print(np.unique(y, return_counts=True))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

(array([0, 1, 2]), array([59, 71, 48]))


In [56]:
# 모델 학습
gb_clf = GradientBoostingClassifier(random_state=42, max_depth=3)
gb_clf.fit(X_train, y_train)

# 모델 평가
y_pred = gb_clf.predict(X_train)
print("Train Accuracy :", accuracy_score(y_train, y_pred))
y_pred = gb_clf.predict(X_test)
print("Test Accuracy :", accuracy_score(y_test, y_pred))


# classification_report 클래스별 예측성능
print(classification_report(y_test, y_pred))

Train Accuracy : 1.0
Test Accuracy : 0.9074074074074074
              precision    recall  f1-score   support

           0       0.86      1.00      0.93        19
           1       0.90      0.90      0.90        21
           2       1.00      0.79      0.88        14

    accuracy                           0.91        54
   macro avg       0.92      0.90      0.90        54
weighted avg       0.91      0.91      0.91        54

